In [1]:
from datetime import datetime, timedelta
from pathlib import Path

import polars as pl
from sklearn.datasets import make_regression

from timelake import TimeLake
from timelake.constants import StorageType

In [2]:
# Params
n_hours = 24 * 30  # 30 days
n_features = 100

# Generate regression features
X, _ = make_regression(n_samples=n_hours, n_features=n_features)
timestamps = [datetime(2024, 1, 1) + timedelta(hours=i) for i in range(n_hours)]

# Create Polars DataFrame
df = pl.DataFrame(X, schema=[f"feature_{i}" for i in range(n_features)])
df = df.with_columns(pl.Series("date", timestamps))

In [3]:
PATH = Path("./my-test-timelake")

lake = TimeLake.create(
    path=PATH,
    df=df,
    timestamp_column="date",
)
df = lake.read()
df.shape

(720, 103)

In [4]:
lake.read(start_date="2024-01-01", end_date="2024-01-03").shape

(72, 103)

In [5]:
df_update = df[0:10]
lake.upsert(df_update)

In [6]:
df = lake.read()
df.shape

(720, 103)

In [7]:
df

feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,…,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,feature_79,feature_80,feature_81,feature_82,feature_83,feature_84,feature_85,feature_86,feature_87,feature_88,feature_89,feature_90,feature_91,feature_92,feature_93,feature_94,feature_95,feature_96,feature_97,feature_98,feature_99,date,date_day,inserted_at
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,datetime[μs],str,datetime[μs]
0.717283,-0.927444,-0.09301,-1.312804,0.842037,2.603085,-1.266181,-1.039496,0.554538,0.808861,-0.450002,-0.347809,1.673727,-1.242567,-0.52886,0.729386,0.981385,0.302241,0.187988,-0.82144,-1.17309,1.680734,-1.783832,0.908725,-0.963227,-0.509153,-0.13706,0.320992,0.376945,-2.104643,0.207738,1.266918,1.487491,-0.52082,0.1659,1.931507,-0.249384,…,0.394851,-2.234399,0.426571,0.636504,0.418331,-0.399574,-0.346066,-0.827534,0.947883,0.629565,-0.469563,0.293792,-1.539689,-0.167741,-0.690682,0.896917,1.983889,-0.533558,-1.207915,-0.29155,2.353922,0.574847,-0.677372,-0.281597,0.530859,-1.804767,-1.070024,-0.141659,0.411941,1.106741,0.755192,0.333842,-0.635836,0.267522,2024-01-07 12:00:00,"""2024-01-07""",2025-04-17 19:16:02.990358
0.672776,0.08284,0.809638,-0.387685,-1.179256,1.016518,-0.051868,-2.020441,0.974602,-1.132222,-2.094278,0.257003,-0.678811,-2.606857,1.119763,0.26674,-0.91504,0.956428,-1.278973,-0.11882,0.59657,-0.554921,0.727802,0.498291,0.704853,0.480467,-0.373227,-0.736683,0.325418,-0.288082,1.670868,0.007755,-1.894855,-0.533117,0.118234,-0.090535,-1.096048,…,-0.36054,0.599707,-0.18514,0.597902,1.065212,1.562043,0.441079,-1.496844,0.534174,1.546275,-0.768917,-0.624257,-0.195208,-1.389408,-0.618929,0.524741,2.229332,1.482452,-0.447917,-1.20978,0.31633,0.102556,-0.733189,0.071518,-1.616441,-1.630373,2.737953,-0.975653,0.331408,0.592675,0.780225,0.314724,1.107742,-0.214007,2024-01-07 10:00:00,"""2024-01-07""",2025-04-17 19:16:02.990358
0.582985,0.525301,-0.460262,-1.033723,0.039901,1.218631,0.073297,0.670387,-0.026995,0.502492,-0.740711,-0.570701,-0.15558,0.040649,0.096253,0.030819,1.001633,1.109537,-0.795219,1.071757,-0.716782,-0.031091,-1.092521,0.986059,0.48797,-0.980578,0.714117,0.657062,0.225714,0.039913,0.288414,-0.926515,-0.349893,-0.245598,1.370642,1.215546,-0.124528,…,0.693162,0.222896,-1.645741,1.181163,-0.60598,0.453652,0.761324,-0.955365,-1.594625,-0.632462,0.818108,-0.691867,-0.253532,0.761249,1.305066,-1.659124,-1.023715,-0.846476,0.948726,0.463339,-0.104594,-1.894437,-1.132915,-1.049086,0.340471,0.112185,0.274587,-2.314097,0.066234,-0.863102,-0.903477,0.433612,0.734867,1.734457,2024-01-07 16:00:00,"""2024-01-07""",2025-04-17 19:16:02.990358
-0.464245,0.406865,0.705744,0.35297,-0.522427,-0.173414,1.266324,-0.275705,-0.465429,0.281795,-1.375678,-1.0773,0.920195,1.698465,-1.051157,0.618711,0.500659,-1.517481,2.003836,-0.764921,0.047147,0.843566,-0.941408,-0.145398,-0.595072,-1.433903,1.64046,-1.012798,-0.07721,-1.365883,-0.933811,0.131153,-0.876398,-0.608656,-1.328725,-0.533263,0.141534,…,1.313797,0.739449,-0.38452,-0.490547,0.920103,-0.644972,-1.938043,0.935592,1.976037,1.178957,-0.429109,-0.048709,-1.411614,1.766656,-0.362301,0.466294,0.341722,-0.13139,-0.051557,-0.206916,-2.992897,-0.267807,-0.12423,0.640998,-0.463013,1.267758,0.590699,0.266497,-0.461422,-0.976217,0.762062,1.374054,0.517282,0.230314,2024-01-07 22:00:0

# We try to create a timelake on S3


In [13]:
import dotenv

dotenv.load_dotenv()
lake = TimeLake.create(
    path="s3://my-time-lake/test",
    df=df,
    timestamp_column="date",
    storage_type=StorageType.S3,
)

In [17]:
lake.read().shape

(720, 103)